In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers datasets

In [2]:
import random
import pandas as pd
from IPython.display import display, HTML

from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

# Get the data

In [3]:
!mkdir ./data && cd data && wget https://raw.githubusercontent.com/nbertagnolli/counsel-chat/master/data/20200325_counsel_chat.csv

mkdir: cannot create directory ‘./data’: File exists


In [4]:
!ls -hl data

total 6.1M
-rw-r--r-- 1 root root 3.2M Mar 28 11:04 20200325_counsel_chat.csv
-rw-r--r-- 1 root root 2.9M Mar 28 11:04 counselchat-data.csv


In [5]:
df = pd.read_csv("data/20200325_counsel_chat.csv", index_col=0)

In [6]:
n_qs = len(df.questionID.unique())
n_topics = len(df.topic.unique())
print(f"Total number of samples {df.shape[0]}, {n_qs} unique questions on {n_topics} topics")

Total number of samples 2129, 815 unique questions on 31 topics


In [7]:
# standardise spaces
df["questionTitle"] = df.questionTitle.map(lambda x: " ".join(x.split()))
df["questionText"] = df.questionText.map(lambda x: " ".join(x.split()))
df["answerText"] = df.answerText.map(lambda x: " ".join(x.split()))

df["prompt"] = "Answer like a therapist:\n" + df.questionTitle + df.questionText + "\nAnswer: "
df["fullText"] = df.prompt + df.answerText
df.head(3)

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split,prompt,fullText
0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train,Answer like a therapist:\nCan I change my feel...,Answer like a therapist:\nCan I change my feel...
1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train,Answer like a therapist:\nCan I change my feel...,Answer like a therapist:\nCan I change my feel...
2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train,Answer like a therapist:\nCan I change my feel...,Answer like a therapist:\nCan I change my feel...


Let's compute prompt and answer length length in tokens:

In [8]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

def get_length(text):
    return len(tokenizer(text)["input_ids"])

df["prompt_length"] = df.prompt.map(get_length)
df["answer_length"] = df.answerText.map(get_length)
df["full_length"] = df.fullText.map(get_length)

In [9]:
df.describe()

,questionID,upvotes,views,prompt_length,answer_length,full_length
count,2129.000000,2129.000000,2129.000000,2129.000000,2129.000000,2129.000000
mean,346.854861,0.489901,198.604979,84.918271,204.780648,288.648661
std,273.706241,0.942429,300.314280,55.576650,151.817316,165.082355
min,0.000000,0.000000,2.000000,23.000000,2.000000,50.000000
25%,78.000000,0.000000,58.000000,53.000000,106.000000,178.000000
50%,321.000000,0.000000,107.000000,75.000000,164.000000,248.000000
75%,588.000000,1.000000,210.000000,102.000000,252.000000,342.000000
max,884.000000,9.000000,3514.000000,668.000000,1108.000000,1208.000000


In [10]:
for name, group in df.groupby("split"):
    print(f"{name} split contains {len(group)} samples ({len(group.questionID.unique())} unique questions)")

    # bla = group[["questionTitle", "questionID"]].groupby(["questionTitle"]).count()

test split contains 117 samples (39 unique questions)
train split contains 1839 samples (695 unique questions)
val split contains 173 samples (81 unique questions)


In [11]:
df.rename(columns={"answerText":"answer"}, inplace=True)
dataset = DatasetDict(**{
    k: Dataset.from_pandas(df.loc[df.split==k,["prompt", "answer", "topic"]]) for k in df.split.unique()
})

In [12]:
import random
import pandas as pd
from IPython.display import display, HTML
from pprint import pprint

def display_examples(dataset, num_examples=5, mode="pprint"):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    idx = random.sample(list(range(len(dataset))), num_examples)
    texts = [f'{sample["prompt"]}, {sample["answer"]}' for sample in dataset.select(idx)]
    df = pd.DataFrame({"text":texts})
    
    if mode=="df":
        display(HTML(df.to_html()))
    elif mode=="pprint":
        for text in texts:
            pprint(text)
            print()
    else:
        raise ValueError(f"{mode} mode is not supported. Please select one of ['df' 'pprint']")

In [13]:
display_examples(dataset["train"], mode="df")

,text
0,"Answer like a therapist:\nWhy do I always push my boyfriend away?My boyfriend shows affection, but I just push him away. Every time my boyfriend tries to kiss, hug, or touch me I almost always push him away. I'm on birth control and it has killed my sex drive. I love him so much. Why do I do this?\nAnswer: , I sympathize with you! It is actually quite common for one partner to have a higher sex drive than the other, and can lead to discord in the relationship. The good news is, there are ways to deal with this.You may have already answered your question. There are many medications that can alter one's sex drive. If your birth control pills are the cause of this, talk to your doctor about switching to a different brand. You may even consider alternate methods for birth control, such as condoms.However, there are some other things to ask yourself. First, you used the term ""affection"" but then talked about sex. They're not the same thing. There are different kinds of intimacy, and many different ways to show love for someone. A book you might find useful is ""The Five Love Languages"".Defining what ""affection"" means to you, and to him, and discovering all the ways you both can show this towards each other, may help lessen some of your anxiety over this.Humans are hard-wired to need appropriate physical touch. In fact, studies have shown that newborns/premies who can receive skin-on-skin contact with their parent can recover faster than babies who don't receive this touch. It's called Kangaroo Care. Children who have grown up in neglectful environments where they're not hugged and kissed and shown appropriate physical contact can often be unable to show this to their partners and children when they're adults. One question to ask yourself is, were you having intimacy issues before starting the medication? If so, there may be other, deeper issues at play here.For someone who has received the wrong kind of sexual/physical touch (sexual abuse, physical beatings) it can be difficult to allow someone to be physically close to you. A therapist could help you work through this.Sometimes, a person may lose their sexual feelings for their partner due to issues in the relationship. Once those issues are resolved, the sex drive comes back. Examine your relationship and see if either of you are unhappy, if you've been arguing a lot, or if there are any problems in your relationship that need to be addressed.Finally, look for the other ways that the two of you may be intimate in a non-sexual way. You may be doing things for each other that you don't even realize, and noticing these will help you appreciate it more. Take the stress off of the sexual gratification and focus on showing your love in other ways. Some people like to give gifts, do the other person's chores, or do other things they think the person will appreciate.If, after all of this, you still have a discrepancy with one of you having a higher sex drive, there are many articles online that offer advice for couples dealing with this situation that you may find useful. It is a very common issue for a lot of couples."
1,"Answer like a therapist:\nHow does counseling end?How does a counselor decide when to end counseling sessions or to terminate working with a client?\nAnswer: , There are typically three reasons why therapy is terminated:1) Client has met therapy goals2) Client is not progressing 3) Therapist is not a good fit for clientIn order to properly assess whether therapy is helping and what progress is being made, the therapist needs to have ways of consistently checking in with clients sessions-by-session to determine what is helping, what isn't, and where the client is at in relation to their original therapy goals. When a client has met their goals, that is a good time to end counselling sessions unless the client has new goals or simply wants to check-in periodically to make sure that they are still on track (sometimes referred to as relapse prevention). When 

In [14]:
display_examples(dataset["train"])

('Answer like a therapist:\n'
 "I'm scared to tell my family that I'm bisexualMy parents seem okay with "
 'other sexualities, but normally they only talk about being gay. When they do '
 "talk about bisexuality, they say things like “they'll do anything” or things "
 "that make me very uncomfortable because I am bisexual. I don't know if I am "
 'ready to come out to them.\n'
 'Answer: , Coming out to family members can cause a lot of anxiety. However, '
 'although I cannot promise what their reaction will be, the benefit is that '
 'you will no longer have to hide who you are. Perhaps researching or getting '
 'information on bisexuality can help your family understand what it truly is '
 'to be bisexual. Many times, people are just not educated on certain things.')

('Answer like a therapist:\n'
 'Is it okay for my girlfriend to have sex with other men since I can’t '
 'sexually perform?I am currently suffering from erectile dysfunction and have '
 'tried Viagra, Cialis, etc. Nothin

Note: "prompt" and "answer" are kept as separate fields because it will be handy later for proper labeling. Tokens corresponding to "prompt" will be ignored at loss computation as we are interested in generating answers only.

In [15]:
sample = dataset["train"][0]
tokenizer(sample["prompt"], sample["answer"], return_token_type_ids=True)

{'input_ids': [33706, 588, 257, 24636, 25, 198, 6090, 314, 1487, 616, 4203, 286, 852, 28063, 284, 2506, 30, 40, 1101, 1016, 832, 617, 1243, 351, 616, 7666, 290, 3589, 13, 314, 8523, 3993, 290, 314, 466, 2147, 475, 892, 546, 703, 314, 1101, 28063, 290, 703, 314, 6584, 470, 307, 994, 13, 314, 1053, 1239, 3088, 393, 39496, 7341, 13, 314, 1053, 1464, 2227, 284, 4259, 616, 2428, 11, 475, 314, 1239, 651, 1088, 284, 340, 13, 1374, 460, 314, 1487, 616, 4203, 286, 852, 28063, 284, 2506, 30, 198, 33706, 25, 220, 1532, 2506, 6834, 345, 821, 28063, 11, 788, 3863, 345, 761, 284, 1064, 649, 661, 284, 8181, 503, 351, 13, 42338, 11, 262, 1919, 4732, 287, 543, 257, 1048, 3160, 318, 257, 1263, 4588, 287, 2116, 12, 31869, 13, 48059, 11, 345, 460, 467, 2835, 290, 2835, 2111, 284, 1833, 1521, 345, 821, 407, 28063, 11, 788, 467, 736, 284, 262, 976, 4315, 290, 307, 13642, 866, 757, 13, 1858, 389, 867, 40840, 6218, 345, 460, 1064, 287, 1919, 2056, 13, 6674, 1100, 617, 286, 262, 3392, 543, 1181, 326, 645, 1048